# Preparation

To start off, follow these steps:
* From the top menu bar, select **Runtime**.
* Click on **change runtime type**.
* Select GPU (preferably T4) and click **save**.

Now run the cell bellow to confirm if GPU is available:

In [11]:
!nvidia-smi

Mon Aug 11 12:30:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Run the cell bellow to install ultralytics:

In [3]:
!pip -q install ultralytics==8.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour

# Uploading the Dataset

Download the official dataset from the links bellow:

Details:
* File name: dataset_yolo.zip
* File size: 150mb
* Version: 1.0

Links:
* [Kaggle](https://www.kaggle.com/datasets/shahzaibahmad05/logic-gates-typerotation-detection-dataset)
* [Google Drive](https://drive.google.com/uc?export=download&id=1H22YKo60RVP0wAn1gruZzJOcp0HdeSIo
)


* Run the cell bellow.
* A **choose files** button will appear.
* Upload dataset_yolo.zip
* This should print *OK* for all 4 files.

In [7]:
from google.colab import files
files.upload()  # choose dataset_yolo.zip
!unzip -q -o dataset_yolo.zip -d dataset_yolo

from pathlib import Path
for p in ["dataset_yolo/images/train","dataset_yolo/images/val","dataset_yolo/labels/train","dataset_yolo/labels/val"]:
    print(p, "OK" if Path(p).exists() else "MISSING")

# Creating the Training Script

Run the cell bellow to create the training script on the server:

In [5]:
%%writefile train_yolov10.py
import shutil
import zipfile
import re
from pathlib import Path

# --- Hardcoded config ---
DATA_YAML = Path("dataset_yolo/data.yaml")
MODEL = "yolov10s.pt"       # best balance for Colab T4
EPOCHS = 120
IMGSZ = 640
BATCH = -1                  # auto-batch
LR0 = 0.01
PATIENCE = 30
PROJECT = Path("runs/train")
RUN_NAME = "logic_gates_yolov10s"
SEED = 42
SAVE_PERIOD = 5  # <-- save & zip every 5 epochs

# --- light dependency bootstrap (Colab-friendly) ---
from ultralytics import YOLO
from ultralytics.utils import SETTINGS
SETTINGS['wandb'] = False  # disable wandb

def _int_epoch_from_name(p: Path) -> int | None:
    m = re.search(r"epoch(\d+)\.pt$", p.name)
    return int(m.group(1)) if m else None

def _latest_epoch_pt(weights_dir: Path) -> tuple[int | None, Path | None]:
    """Return (latest_epoch_number, path_to_epochN.pt) if present."""
    epoch_files = list(weights_dir.glob("epoch*.pt"))
    if not epoch_files:
        return None, None
    latest = max(epoch_files, key=lambda f: _int_epoch_from_name(f) or -1)
    return _int_epoch_from_name(latest), latest

def _zip_checkpoint(weights_dir: Path, epoch_number: int, out_dir: Path):
    """
    Create a zip named after the epoch containing:
      - epoch{N}.pt (if exists),
      - last.pt,
      - best.pt (if exists),
      - selected run metadata (results.csv, args/hyp files if present).
    """
    out_dir.mkdir(parents=True, exist_ok=True)
    zip_path = out_dir / f"checkpoint_epoch_{epoch_number}.zip"
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        # Core weights
        ep = weights_dir / f"epoch{epoch_number}.pt"
        if ep.exists():
            zf.write(ep, arcname=f"weights/epoch{epoch_number}.pt")
        last = weights_dir / "last.pt"
        if last.exists():
            zf.write(last, arcname="weights/last.pt")
        best = weights_dir / "best.pt"
        if best.exists():
            zf.write(best, arcname="weights/best.pt")

        # Helpful run artifacts if available
        run_root = weights_dir.parent  # runs/train/<name>
        for cand in ["results.csv", "hyp.yaml", "args.yaml", "opt.yaml", "metrics.json"]:
            p = run_root / cand
            if p.exists():
                zf.write(p, arcname=f"run/{p.name}")

    print(f"📦 Saved: {zip_path}")
    return zip_path

def main():
    if not DATA_YAML.exists():
        raise SystemExit(f"data.yaml not found at {DATA_YAML}. Run prepare_dataset.py first.")

    print(f"Loading model: {MODEL}")
    model = YOLO(MODEL)

    # Where outputs land
    run_dir = PROJECT / RUN_NAME
    weights_dir = run_dir / "weights"
    zips_dir = run_dir / "checkpoints_zips"

    epochs_done = 0
    # Train in SAVE_PERIOD chunks so we can zip after each window
    while epochs_done < EPOCHS:
        chunk = min(SAVE_PERIOD, EPOCHS - epochs_done)
        print(f"\n=== Training chunk: epochs {epochs_done} -> {epochs_done + chunk} (size={chunk}) ===")

        # First chunk: fresh; subsequent chunks: resume
        resume_flag = epochs_done > 0

        results = model.train(
            data=str(DATA_YAML),
            epochs=chunk,
            imgsz=IMGSZ,
            batch=BATCH,
            lr0=LR0,
            patience=PATIENCE,
            project=str(PROJECT),
            name=RUN_NAME,
            exist_ok=True,
            pretrained=True,
            seed=SEED,
            deterministic=False,
            single_cls=False,

            # Efficiency / stability
            rect=False,         # allow mosaic
            cos_lr=True,
            amp=True,
            device="0",
            workers=2,          # safer on Colab
            cache="ram",

            # SAFE augmentations (do not change orientation)
            hsv_h=0.015, hsv_s=0.5, hsv_v=0.4,
            translate=0.08, scale=0.40,
            mosaic=0.50,        # moderate mosaic; reduce if VRAM tight
            close_mosaic=10,

            # CRITICAL: disable orientation-changing augs
            degrees=0.0, shear=0.0, perspective=0.0, flipud=0.0, fliplr=0.0,

            optimizer="AdamW",
            plots=True,
            save_period=SAVE_PERIOD,  # will still emit epochN.pt within chunk if N hits the boundary
            resume=resume_flag,
        )

        # After this chunk, locate the latest epochN.pt and zip state under that epoch number
        latest_n, latest_ep_path = _latest_epoch_pt(weights_dir)
        if latest_n is not None:
            _zip_checkpoint(weights_dir, latest_n, zips_dir)
        else:
            print("[warn] No epoch*.pt found after this chunk; skipping zip.")

        epochs_done += chunk

    print("✅ Training finished.")

    # Convenience copies at project root (optional)
    best_src = weights_dir / "best.pt"
    if best_src.exists():
        shutil.copy2(best_src, "best_model.pt")
        print("Saved: best_model.pt")
    else:
        print(f"[warn] best.pt not found at {best_src}")

    # Optional quick val
    _ = model.val(data=str(DATA_YAML), imgsz=IMGSZ)

if __name__ == "__main__":
    main()


Writing train_yolov10.py


# Important Step: Training

✔️ Quick Notes:
* The training script would download a zip file every 5 epochs. Please keep it saved.
* If the training is interrupted, resume it from the last save point. For guidance on this, check out **Resuming Section**.
* This prevents progress loss due to interruptions.
* Training may take upto 5–7 hours. Please be patient.

Run the cell bellow to start training:

In [4]:
!python train_yolov10.py

python3: can't open file '/content/train_yolov10.py': [Errno 2] No such file or directory


# Restore: Resuming when Interrupted

Follow these steps if the training was interrupted:
* Make sure to run the cells in **Preparation** and **Uploading the Dataset** sections.
* Then follow these steps bellow:

* **Run** the cell bellow.
* An **upload button** should appear.
* Upload the zip file **last saved** by the training script. It would have been saved in your downloads folder as checkpoint_epoch_XX.zip

In [ ]:
from google.colab import files
up = files.upload()  # pick your checkpoint_epoch_XX.zip
zip_name = next(iter(up.keys()))
print("Uploaded:", zip_name)

Run the cell bellow to **unpack the files** into the run directory:

In [ ]:
from pathlib import Path
import zipfile

PROJECT = Path("/content/runs/train")
RUN_NAME = "logic_gates_yolov10s"
run_dir = PROJECT / RUN_NAME
run_dir.mkdir(parents=True, exist_ok=True)

with zipfile.ZipFile(zip_name, "r") as zf:
    zf.extractall(run_dir)

# sanity check
print("Extracted to:", run_dir)
!ls -lah "/content/runs/train/logic_gates_yolov10s/weights"

Resume training by running the following cell:

In [ ]:
from ultralytics import YOLO

last_ckpt = run_dir / "weights" / "last.pt"
assert last_ckpt.exists(), f"Missing: {last_ckpt}"

model = YOLO(str(last_ckpt))
# resume=True restores optimizer, epoch count, etc.
model.train(
    resume=True,
    project=str(PROJECT),
    name=RUN_NAME,
)

# Finishing

After the training is completed, run the cell bellow to download the trained model.

In [ ]:
from google.colab import files
files.download('/content/runs/train/logic_gates_yolov10s/weights/best_model.pt')